In [1]:
!pip install requests pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 159.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 181.5 kB/s eta 0:00:00a 0:00:01


In [3]:
import socket
print(socket.gethostbyname('google.com'))

216.58.204.142


In [33]:
import requests

# Wir nutzen den TAWES-Endpunkt, der für Station 11035 meist offen ist
url = "https://dataset.api.hub.geosphere.at/v1/station/historical/tawes-v1-10min"

params = {
    "parameters": "TL",         # Lufttemperatur (10-Minuten-Takt)
    "station_ids": "11035",     # Wien Hohe Warte
    "start": "2024-01-01T00:00",
    "end": "2024-01-10T02:00",  # Zeitraum zum Testen kurz halten
    "format": "geojson"         # GeoJSON ist das Standard-Datenformat der API
} 

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    print("Erfolg! Daten geladen.")
    # Zeige die ersten paar Zeitstempel und Werte
    timestamps = data.get("timestamps", [])
    values = data["features"][0]["properties"]["parameters"]["TL"]["data"]
    for ts, val in zip(timestamps[:5], values[:5]):
        print(f"Zeit: {ts} -> Tempe: {val}°C")
else:
    print(f"Fehler: {response.status_code}")
    print(response.text)

Erfolg! Daten geladen.
Zeit: 2024-01-01T00:00+00:00 -> Tempe: None°C
Zeit: 2024-01-01T00:10+00:00 -> Tempe: None°C
Zeit: 2024-01-01T00:20+00:00 -> Tempe: None°C
Zeit: 2024-01-01T00:30+00:00 -> Tempe: None°C
Zeit: 2024-01-01T00:40+00:00 -> Tempe: None°C


In [26]:
import requests

# Echtzeit-Monitor Endpunkt
url = "https://www.wienerlinien.at/ogd_realtime/monitor"

params = {
    "rbl": "4426", # Stephansplatz U3
    "activateTrafficInfo": "true"
}

print("Rufe Wiener Linien Echtzeitdaten ab...")
response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    monitors = data["data"]["monitors"]
    print(f"Haltestelle: {monitors[0]['locationStop']['properties']['title']}")
    
    for line in monitors[0]["lines"]:
        departure = line["departures"]["departure"][0]["departureTime"]["countdown"]
        print(f"Linie {line['name']} Richtung {line['towards']} in {departure} Minuten.")
else:
    print(f"Fehler: {response.status_code}")

Rufe Wiener Linien Echtzeitdaten ab...
Haltestelle: Schönbrunn
Linie U4 Richtung HÜTTELDORF        in 3 Minuten.


In [37]:
import requests
import io
import pandas as pd # Falls installiert, ansonsten nutzen wir csv Modul

url = "https://dataset.api.hub.geosphere.at/v1/station/historical/tawes-v1-10min"

# Wir wählen einen Zeitraum, der garantiert existiert (z.B. Frühling 2023)
params = {
    "parameters": "TL",
    "station_ids": "11035",
    "start": "2023-05-01T00:00",
    "end": "2023-05-01T05:00",
    "format": "csv" # CSV ist robuster für historische Daten
}

response = requests.get(url, params=params)

if response.status_code == 200:
    if response.text.strip() == "":
        print("Die API hat eine leere Datei geliefert. Versuche einen anderen Zeitraum.")
    else:
        print("Erfolg! Hier sind die Rohdaten:\n")
        print(response.text)
        
        # Optional: Direkt in ein Pandas DataFrame laden für die Analyse
        # df = pd.read_csv(io.StringIO(response.text))
        # print(df.head())
else:
    print(f"Fehler: {response.status_code}")
    print("Meldung:", response.text)

Erfolg! Hier sind die Rohdaten:

{"media_type":"application/json","type":"FeatureCollection","version":"v1","timestamps":["2023-05-01T00:00+00:00","2023-05-01T00:10+00:00","2023-05-01T00:20+00:00","2023-05-01T00:30+00:00","2023-05-01T00:40+00:00","2023-05-01T00:50+00:00","2023-05-01T01:00+00:00","2023-05-01T01:10+00:00","2023-05-01T01:20+00:00","2023-05-01T01:30+00:00","2023-05-01T01:40+00:00","2023-05-01T01:50+00:00","2023-05-01T02:00+00:00","2023-05-01T02:10+00:00","2023-05-01T02:20+00:00","2023-05-01T02:30+00:00","2023-05-01T02:40+00:00","2023-05-01T02:50+00:00","2023-05-01T03:00+00:00","2023-05-01T03:10+00:00","2023-05-01T03:20+00:00","2023-05-01T03:30+00:00","2023-05-01T03:40+00:00","2023-05-01T03:50+00:00","2023-05-01T04:00+00:00","2023-05-01T04:10+00:00","2023-05-01T04:20+00:00","2023-05-01T04:30+00:00","2023-05-01T04:40+00:00","2023-05-01T04:50+00:00","2023-05-01T05:00+00:00"],"features":[{"type":"Feature","geometry":{"type":"Point","coordinates":[16.35638888888889,48.248611111

In [54]:
import requests
import json

# Die Basis-URL (Endpunkt)
base_url = "https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v1-10min"

# Die Parameter als Dictionary (Übersichtlicher und leichter zu ändern)
params = {
    "parameters": ["RR", "TL"],      # Niederschlag (RR) und Temperatur (TL)
    "start": "2024-01-01T00:00",
    "end": "2024-01-10T00:00",
    "station_ids": "5882",           # Die gewünschte Stations-ID
    "output_format": "geojson"       # Das funktionierende Format
}

print("Sende Anfrage an GeoSphere API...")

try:
    # Der API-Call
    response = requests.get(base_url, params=params)
    
    # Prüfen, ob die Anfrage erfolgreich war (Status 200)
    if response.status_code == 200:
        data = response.json()
        print("ERFOLG: Daten empfangen!")
        
        # Kurze Analyse der Struktur
        if "features" in data and len(data["features"]) > 0:
            count = len(data.get("timestamps", []))
            print(f"Anzahl der Zeitstempel: {count}")
            
            # Beispiel: Ersten Temperaturwert anzeigen
            first_temp = data["features"][0]["properties"]["parameters"]["TL"]["data"][0]
            print(f"Erste gemessene Temperatur: {first_temp}°C")

        
        else:
            print("Verbindung ok, aber keine Daten im 'features' Feld gefunden.")
            
    elif response.status_code == 403:
        print("Fehler 403: Zugriff verweigert. Prüfe Stations-ID oder Parameter.")
    else:
        print(f"Fehler {response.status_code}: {response.text}")

except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {e}")

Sende Anfrage an GeoSphere API...
ERFOLG: Daten empfangen!
Anzahl der Zeitstempel: 1297
Erste gemessene Temperatur: 6.5°C


In [58]:
import requests
import json
import pandas as pd

# Die Basis-URL (Klima-Endpunkt)
base_url = "https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v1-10min"

# Alle deine gewünschten Parameter
params = {
    "parameters": ["RR", "TL", "P", "FF", "SO", "RF"],
    "start": "2024-01-01T00:00",
    "end": "2024-01-02T00:00", # Zeitraum kurz für den Test
    "station_ids": "5882",
    "output_format": "geojson"
}

print("Sende Anfrage an GeoSphere API...")

try:
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        print("ERFOLG: Daten empfangen!")
        
        timestamps = data.get("timestamps", [])
        features = data.get("features", [])

        if features:
            props = features[0]["properties"]["parameters"]
            station_id = features[0]["properties"].get("station", "5882")
            
            # 1. Daten in ein Dictionary sammeln
            storage_dict = {"timestamp": timestamps}
            
            for p in params["parameters"]:
                if p in props:
                    storage_dict[p] = props[p]["data"]
            
            # 2. In DataFrame umwandeln
            df = pd.DataFrame(storage_dict)
            
            # Ersten Wert anzeigen (wie gewünscht)
            if not df.empty and "TL" in df.columns:
                print(f"Erster gemessener Temperaturwert: {df['TL'].iloc[0]}°C")
            
            # 3. Vorbereitung für MongoDB (Liste von Dokumenten)
            # Wir fügen die Stations-ID zu jedem Datensatz hinzu
            df['station_id'] = station_id
            mongo_docs = df.to_dict(orient="records")
            
            print(f"Frame erstellt. {len(mongo_docs)} Datensätze bereit für MongoDB.")
            
            # Beispiel-Ausgabe eines Datensatzes für MongoDB
            print("\nBeispiel-Datensatz für MongoDB:")
            print(json.dumps(mongo_docs[0], indent=2))
            
            # HIER könntest du jetzt schreiben:
            # collection.insert_many(mongo_docs)
            
        else:
            print("Verbindung ok, aber keine Features gefunden.")
    else:
        print(f"Fehler {response.status_code}: {response.text}")

except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {e}")

Sende Anfrage an GeoSphere API...
ERFOLG: Daten empfangen!
Erster gemessener Temperaturwert: 6.5°C
Frame erstellt. 145 Datensätze bereit für MongoDB.

Beispiel-Datensatz für MongoDB:
{
  "timestamp": "2024-01-01T00:00+00:00",
  "RR": 0.0,
  "TL": 6.5,
  "P": 983.8,
  "FF": 1.5,
  "SO": 0.0,
  "RF": 71.0,
  "station_id": "5882"
}


In [61]:
import requests
import pandas as pd
from pymongo import MongoClient, errors
import io

# --- KONFIGURATION ---
DB_NAME = "BDA"        # So heißt deine Datenbank
COLLECTION_NAME = "Station_5882" # So heißt deine Tabelle (Collection)

# API Setup
base_url = "https://dataset.api.hub.geosphere.at/v1/station/historical/klima-v1-10min"
params = {
    "parameters": ["RR", "TL", "P", "FF", "SO", "RF"],
    "start": "2024-01-01T00:00",
    "end": "2024-12-31T00:00",
    "station_ids": "5882",
    "output_format": "geojson"
}

try:
    # 1. Daten von GeoSphere laden
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    data = response.json()
    
    # DataFrame erstellen
    timestamps = data.get("timestamps", [])
    props = data["features"][0]["properties"]["parameters"]
    storage_dict = {"timestamp": timestamps}
    for p in params["parameters"]:
        if p in props:
            storage_dict[p] = props[p]["data"]
    
    df = pd.DataFrame(storage_dict)
    df['station_id'] = "5882"
    mongo_docs = df.to_dict(orient="records")

    # --- 2. MONGODB VERBINDUNG & INDEX ---
    # Nutze den Namen, der in deinem Docker-Screenshot steht
    client = MongoClient('mongodb://big_data_mongo:27017/')
    db = client[DB_NAME]
    collection = db[COLLECTION_NAME]

    # UNIQUE INDEX ERSTELLEN
    # 'timestamp' muss einzigartig sein. 1 bedeutet aufsteigend sortiert.
    print("Erstelle Unique Index für 'timestamp'...")
    collection.create_index([("timestamp", 1)], unique=True)

    # --- 3. DATEN SPEICHERN MIT DUPLIKAT-CHECK ---
    if mongo_docs:
        success_count = 0
        duplicate_count = 0
        
        for doc in mongo_docs:
            try:
                collection.insert_one(doc)
                success_count += 1
            except errors.DuplicateKeyError:
                # Dieser Fehler tritt auf, wenn der Timestamp schon existiert
                duplicate_count += 1
        
        print(f"--- ABSCHLUSSBERICHT ---")
        print(f"Datenbank: {DB_NAME} | Collection: {COLLECTION_NAME}")
        print(f"Neu gespeichert: {success_count}")
        print(f"Übersprungene Duplikate: {duplicate_count}")
    
except Exception as e:
    print(f"Fehler: {e}")

Erstelle Unique Index für 'timestamp'...
--- ABSCHLUSSBERICHT ---
Datenbank: BDA | Collection: Station_5882
Neu gespeichert: 35280
Übersprungene Duplikate: 0


In [1]:
import requests
import pandas as pd
from pymongo import MongoClient, errors
import io

# --- KONFIGURATION ---
DB_NAME = "big_data_austria"        # So heißt deine Datenbank
COLLECTION_NAME = "Station_HW_25" # So heißt deine Tabelle (Collection)

# API Setup
base_url = "https://dataset.api.hub.geosphere.at/v1/station/historical/tawes-v1-10min"
params = {
    "parameters": ["RR", "TL", "P", "FF", "SO", "RF"],
    "start": "2025-11-01T00:00",
    "end": "2025-12-31T00:00",
    "station_ids": "11035",
    "output_format": "geojson"
}

try:
    # 1. Daten von GeoSphere laden
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    data = response.json()
    
    # DataFrame erstellen
    timestamps = data.get("timestamps", [])
    props = data["features"][0]["properties"]["parameters"]
    storage_dict = {"timestamp": timestamps}
    for p in params["parameters"]:
        if p in props:
            storage_dict[p] = props[p]["data"]
    
    df = pd.DataFrame(storage_dict)
    df['station_id'] = "HW_25"
    mongo_docs = df.to_dict(orient="records")

    # --- 2. MONGODB VERBINDUNG & INDEX ---
    # Nutze den Namen, der in deinem Docker-Screenshot steht
    client = MongoClient('mongodb://mongodb:27017')
    db = client[DB_NAME]
    collection = db[COLLECTION_NAME]

    # UNIQUE INDEX ERSTELLEN
    # 'timestamp' muss einzigartig sein. 1 bedeutet aufsteigend sortiert.
    print("Erstelle Unique Index für 'timestamp'...")
    collection.create_index([("timestamp", 1)], unique=True)

    # --- 3. DATEN SPEICHERN MIT DUPLIKAT-CHECK ---
    if mongo_docs:
        success_count = 0
        duplicate_count = 0
        
        for doc in mongo_docs:
            try:
                collection.insert_one(doc)
                success_count += 1
            except errors.DuplicateKeyError:
                # Dieser Fehler tritt auf, wenn der Timestamp schon existiert
                duplicate_count += 1
        
        print(f"--- ABSCHLUSSBERICHT ---")
        print(f"Datenbank: {DB_NAME} | Collection: {COLLECTION_NAME}")
        print(f"Neu gespeichert: {success_count}")
        print(f"Übersprungene Duplikate: {duplicate_count}")
    
except Exception as e:
    print(f"Fehler: {e}")

Erstelle Unique Index für 'timestamp'...
--- ABSCHLUSSBERICHT ---
Datenbank: big_data_austria | Collection: Station_HW_25
Neu gespeichert: 8641
Übersprungene Duplikate: 0
